# LangChain Expression Language (LCEL)

In [1]:
import openai
import os

In [2]:
openai.api_key = open("openAI_key.txt", "r").read().strip("\n")

In [3]:
#!pip install pydantic==1.10.9
#!pip install langchain==0.0.336
#!pip install docarray==0.39.1  

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [5]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(openai_api_key=openai.api_key)
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(openai_api_key=openai.api_key)
)
retriever = vectorstore.as_retriever()

AttributeError: 'builtin_function_or_method' object has no attribute '__func__'

In [ ]:
retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap

In [12]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [13]:
chain.invoke({"question": "where did harrison work?"})

NameError: name 'retriever' is not defined

In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})

## Bind

and OpenAI Functions

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0, openai_api_key=openai.api_key).bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [ ]:
model = model.bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

## Fallbacks

In [ ]:
from langchain.llms import OpenAI
import json

In [ ]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001", 
    openai_api_key=openai.api_key
)
simple_chain = simple_model | json.loads

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [ ]:
simple_model.invoke(challenge)

Note: The next line is expected to fail.

In [ ]:
simple_chain.invoke(challenge)

In [ ]:
model = ChatOpenAI(temperature=0, openai_api_key=openai.api_key)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

## Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(openai_api_key=openai.api_key)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

In [ ]:
for t in chain.stream({"topic": "bears"}):
    print(t)

In [ ]:
response = await chain.ainvoke({"topic": "bears"})
response

In [ ]:
response = await chain.ainvoke({"topic": "bunnies"})
response

In [ ]:
response = await chain.ainvoke({"topic": "dogs"})
response

In [ ]:
response = await chain.ainvoke({"topic": "crocodiles"})
response

In [ ]:
response = await chain.ainvoke({"topic": "gingerbread man"})
response